# USE HMM network to inference 

1. Since max step =2, each room can generate a HMM with neibours room in previous time state and relevant sensors at current and all previous  time stamp.
2. we use a binary situation for outcome space {0,1} 
3. Based on the network, we will learn P(X_1={0/1} | X ={0/1}}, note the state will determine the action directly
4. Then we can inference B(X_t) by passage of time and observations Topic 8 from page 28


In [111]:
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [112]:
data = pd.read_csv("data.csv")

In [113]:
data.columns

Index(['Unnamed: 0', 'reliable_sensor1', 'reliable_sensor2',
       'reliable_sensor3', 'reliable_sensor4', 'unreliable_sensor1',
       'unreliable_sensor2', 'unreliable_sensor3', 'unreliable_sensor4',
       'robot1', 'robot2', 'door_sensor1', 'door_sensor2', 'door_sensor3',
       'door_sensor4', 'time', 'electricity_price', 'r1', 'r2', 'r3', 'r4',
       'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15',
       'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25',
       'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35',
       'c1', 'c2', 'c3', 'c4', 'o1', 'outside'],
      dtype='object')

In [114]:
data_copy = copy.deepcopy(data)

In [115]:
MAP = {
    "r1":["r2","r3"],
    "r2":["r1","r4"],
    "r3":["r1","r7"],
    "r4":["r2","r8"],
    "r5":["r6","r9","c3"],
    "r6":["r5","c3"],
    "r7":["r3","c1"],
    "r8":["r4","r9"],
    "r9":["r5","r8","r13"],
    "r10":["c3"],
    "r11":["c3"],
    "r12":["outside","r22"],
    "r13":["r9","r24"],
    "r14":["r24"],
    "r15":["c3"],
    "r16":["c3"],
    "r17":["c3"],
    "r18":["c3"],
    "r19":["c3"],
    "r20":["c3"],
    "r21":["c3"],
    "r22":["r12","r25"],
    "r23":["r24"],
    "r24":["r13","r14","r23"],
    "r25":["r22","r26","c1"],
    "r26":["r25","r27"],
    "r27":["r26","r32"],
    "r28":["c4"],
    "r29":["c4","r30"],
    "r30":["r29"],
    "r31":["r32"],
    "r32":["r27","r31","r33"],
    "r33":["r32"],
    "r34":["c2"],
    "r35":["c4"],
    "c1":["r7","r25","c2"],
    "c2":["c1","r34","c4"],
    "c3": ["r5","r6","r10","r11","r15","r16","r17","r18","r19","r20","r21","o1"],
    "c4":["r29","c2","r35","r28","o1"],
    "o1":["c3","c4"],
    "outside":["r12"]  
}
censor_dict =  {
    "r1": "us3", #us for unreliable_sensor
    "r5": "rs2" ,#rs for reliable_sensor
    "r8": "ds1", #ds for door sensor
    "r9": "ds1",
    "r16": "rs1",
    "r24" : "us4",
    "r25" : "rs3",
    "r26" : "ds3", 
    "r27" : "ds3",
    "r31" : "rs4",
    "r35" : "ds4",
    "c1" : "ds2",
    "c2" : "ds2",
    "c3" : "us2",
    "c4" : "ds4",
    "o1" : "us1"
}

In [116]:
def n_step_neighbour(node,n,G):
    neighbour_list = []
    neighbour_list = neighbour_list + G[node]
    while n > 1:
        for new_node in neighbour_list:
            neighbour_list = neighbour_list+ G[new_node]
        n = n -1 
    n_list = list(set(neighbour_list))
    n_list.remove(node)
    return(n_list)

In [117]:
def generate_HMM(node,G,censors):
    HMM = {}
    curr_node = "current_" + node
    node_neighbour = n_step_neighbour(node,2,G)
    HMM = {i:[curr_node] for i in node_neighbour}
    HMM[node] = [curr_node]
    
    related_censor = [censors.get(node)] + [censors.get(i) for i in node_neighbour]
    related_censor = [x for x in related_censor  if x is not None]
    
    for i in related_censor:
        if i[0] =="d":
            if sum([censor == i for censor in related_censor]) < 2:
                related_censor.remove(i) #drop door censor when you only at one side
    
    
    
    HMM[curr_node] = list(set(related_censor))
    for censor_evi in related_censor: 
        HMM[censor_evi] = []
    
    return(HMM)
    
    
#def learn_prob(G,data):
    

In [118]:
generate_HMM('r25',MAP,censor_dict)


{'r27': ['current_r25'],
 'r7': ['current_r25'],
 'r26': ['current_r25'],
 'c1': ['current_r25'],
 'r22': ['current_r25'],
 'r12': ['current_r25'],
 'c2': ['current_r25'],
 'r25': ['current_r25'],
 'current_r25': ['rs3', 'ds3', 'ds2'],
 'rs3': [],
 'ds3': [],
 'ds2': []}

In [119]:
def learn_tranisition(curr_node,pre_node,data):
    new_df = pd.DataFrame()
    new_df['curr'] = data[curr_node][1:]
    new_df['pre'] = data[pre_node].shift(1)[:-1]
    
    
    prob_ct = pd.crosstab(new_df['pre'] > 0 , new_df['curr'] > 0, normalize = "index")
    
    tran_table = odict()
    
    tran_table[(1,1)] = prob_ct.loc[True,True] # p(current = 1 | previous = 1)
    tran_table[(1,0)] = prob_ct.loc[True,False]
    tran_table[(0,1)] =prob_ct.loc[False,True]# p(current = 1 | previous = 0)
    tran_table[(0,0)] =  prob_ct.loc[False,False]
    
    
    return ({'dom': (pre_node,curr_node), 'table':tran_table})

def learn_censor_prob(node, censor, data):
    
    prob_ct = pd.crosstab(data[node] > 0 , data[censor] == "motion", normalize = "index")
    
    tran_table = odict()
    
    tran_table[(1,1)] = prob_ct.loc[True,True] # p(current = 1 | previous = 1)
    tran_table[(1,0)] = prob_ct.loc[True,False]
    tran_table[(0,1)] =prob_ct.loc[False,True]# p(current = 1 | previous = 0)
    tran_table[(0,0)] =  prob_ct.loc[False,False]
    
    return ({'dom': (node,censor), 'table':tran_table})

In [120]:
learn_tranisition('r22','r25',data)

{'dom': ('r25', 'r22'),
 'table': OrderedDict([((1, 1), 0.10268378063010501),
              ((1, 0), 0.897316219369895),
              ((0, 1), 0.13933895009721323),
              ((0, 0), 0.8606610499027868)])}

In [121]:
learn_censor_prob('r25','reliable_sensor3',data)

{'dom': ('r25', 'reliable_sensor3'),
 'table': OrderedDict([((1, 1), 0.9568261376896149),
              ((1, 0), 0.043173862310385065),
              ((0, 1), 0.03367875647668394),
              ((0, 0), 0.966321243523316)])}

For one room:
    1. generate the network 
    2. Assign the conditional prob (factor tables) 
    3. Find P(X=1| other nodes, e) and P(X=0| other nodes, e) by HMM algorithm;
    4. Make decicions (threshold) 
 
Entire inference process: 
    1. calculate all the probabilities 
    2. For t = 1, ... T: 
        a. Do inference for each room
        b. Store the decision 
        c. move to next timestep 
             
  
  

In [137]:
# room 25 P(x= 1 | A) and P(x= 0|A)
rooms_df = pd.read_csv("rooms_tran3.csv")
censor_df= pd.read_csv("censor_prob2.csv")


In [138]:
room = 'c1'
room_prob = rooms_df.loc[rooms_df['current_room'] == room, :]
censor_prob = censor_df.loc[censor_df['room'] == room, :] #extract relevant probabilities 



Next step
1. reset index for room_prob and censor_prob
2. extract the probabilities from the dataset
3. compute the P(x=1) and P(x=0) with conditions
4. log likelihood sum 
5. return the prob of room 

In [139]:
def prob_have_people(state,trans, room):
    "trans is the dictionary from rooms_tran/ censor trans; state is a dictionary with room as key and (p(0), p(1)) as value "
    prob_have = trans['(1, 1)']*state[room][1] +  trans["(1, 0)"]*state[room][0]
    return (prob_have)
    
def prob_no_people(state,trans, room):    
    prob_no = trans["(0, 1)"]*state[room][1] +  trans["(0, 0)"]*state[room][0] 
    return (prob_no)

room_prob_tem = room_prob.set_index('previous_room')
state_tem = {'c2': (1.0,0)}
trans_tem = room_prob_tem.loc['c2',:].to_dict()
#print(trans_tem)

print(prob_have_people(state_tem, trans_tem,"c2"))

print(prob_no_people(state_tem, trans_tem,"c2"))

0.2056843679880329
0.3518344308560677


In [140]:
def joint_prob_room(curr_room, room_prob_df, state):
    list_have_people = []
    list_no_people = []
    room_prob_tem =room_prob_df.set_index('previous_room')
    for rooms in room_prob_df['previous_room']:
        room_dict = room_prob_tem.loc[rooms,:].to_dict()
        have_people = prob_have_people(state,room_dict, rooms)
        no_people =  prob_no_people(state,room_dict, rooms)
        list_have_people.append(have_people)
        list_no_people.append(no_people)
    
    prob_have = np.prod(list_have_people)
    prob_no = np.prod(list_no_people)
    
    prob_have = prob_have/ (prob_have+ prob_no)
    prob_no = 1- prob_have  
    
    return(( prob_no,prob_have))
    

In [141]:
state_tem = {room : (0.8,0.2) for room in room_prob['previous_room'] }
print(state_tem)

probability_room = joint_prob_room(room, room_prob, state_tem)
probability_room

{'c4': (0.8, 0.2), 'r3': (0.8, 0.2), 'r25': (0.8, 0.2), 'r34': (0.8, 0.2), 'r26': (0.8, 0.2), 'r22': (0.8, 0.2), 'r7': (0.8, 0.2), 'c2': (0.8, 0.2), 'c1': (0.8, 0.2)}


(0.9470487486017466, 0.05295125139825336)

1. censors 
2. combine the prob
3. iterate all rooms in one timestep 
4. all rooms all time 


In [142]:
def censor_cal(curr_room, censor_prob_df, state):
    list_have_people = []
    list_no_people = []
    censor_prob_tem = censor_prob_df.set_index('censor')
    for censors in censor_prob_df['censor']:
        censor_dict = censor_prob_tem.loc[censors,:].to_dict()
        have_people = prob_have_people(state, censor_dict, censors )
        no_people = prob_no_people(state, censor_dict, censors )
        list_have_people.append(have_people)
        list_no_people.append(no_people)
    
    prob_have = np.prod(list_have_people)
    prob_no = np.prod(list_no_people)

    
    prob_have = prob_have/ (prob_have+ prob_no)
    prob_no = 1- prob_have  
    return(( prob_no, prob_have))
   
    

In [143]:
state_temp = {'reliable_sensor3': (0.0,1.0)} 
probability_censor = censor_cal('c1',censor_prob,state_temp)
probability_censor

(0.4925900982912593, 0.5074099017087407)

In [144]:
def joint_prob(prob_room, prob_censor, weight):
    no_people = prob_room[0] * weight[0] * prob_censor[0] * weight[1]
    have_people = prob_room[1] * weight[0] * prob_censor[1] * weight[1]
    
    prob_have = have_people/ (have_people+ no_people)
    prob_no = 1- prob_have 
    
    return(( prob_no, prob_have))

def make_decision(prob_room, prob_censor, weight, theta):
    people = 0
    no_people = prob_room[0] * weight[0] * prob_censor[0] * weight[1]
    have_people = prob_room[1] * weight[0] * prob_censor[1] * weight[1]
    
    prob_have = have_people/ (have_people+ no_people)
    if prob_have > theta: 
        people = 1
    return(people)
    
make_decision(probability_room, probability_censor, [1,9], 0.5)    

0

Now let's do all rooms at one time stamp given previous time step

In [145]:
data_sample = data.sample(1)
censor_sample = data_sample.iloc[:,1:9]
room_sample = data_sample.iloc[:,17:]

In [146]:
room_dict = room_sample.to_dict()
room_state = {k: list(v.values())[0] for k, v in room_dict.items()}
room_state2 = {k: (0,1) if v > 0 else (1,0) for k,v in room_state.items() }

censor_dict = censor_sample.to_dict()
censor_state = {k: list(v.values())[0] for k, v in censor_dict.items()}
censor_state2 = {k: (0,1) if v == "motion" else (1,0) for k,v in censor_state.items() }
censor_state2

{'reliable_sensor1': (1, 0),
 'reliable_sensor2': (0, 1),
 'reliable_sensor3': (0, 1),
 'reliable_sensor4': (0, 1),
 'unreliable_sensor1': (1, 0),
 'unreliable_sensor2': (1, 0),
 'unreliable_sensor3': (1, 0),
 'unreliable_sensor4': (1, 0)}

In [148]:
def dict_to_state(data_dict, room_info):
    if room_info == "room":
        data_state = {k: list(v.values())[0] for k, v in data_dict.items()}
        data_state2 = {k: (0,1) if v > 0 else (1,0) for k,v in data_state.items() }
    if room_info == "censor":
        #data_state = {k: list(v.values())[0] for k, v in data_dict.items()}
        data_state2= {k: (0,1) if v == "motion" else (1,0) for k,v in data_dict.items() }
    return(data_state2)

In [149]:
def all_room_predict(room_state, censors_state, room_prob_df, censor_prob_df, w, theta = 0.5):
    room_predict_dict = {}
    room_list = list(room_state.keys())
    room_list = [room for room in room_list]
    for room in room_list: 
        room_prob = room_prob_df.loc[room_prob_df['current_room'] == room, :]
        censor_prob =  censor_prob_df.loc[censor_prob_df['room'] == room, :]
        probability_room = joint_prob_room(room, room_prob, room_state)
        probability_censor = censor_cal(room,censor_prob,censors_state)
        room_predict_dict[room] = joint_prob(probability_room , probability_censor, w )
        
    return room_predict_dict

In [150]:
all_room_predict(room_state2,censor_state2, rooms_df,censor_df, [1,1]  )

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


{'r1': (0.9974229512106813, 0.0025770487893187243),
 'r2': (0.723657410068943, 0.276342589931057),
 'r3': (0.06566990405806616, 0.9343300959419338),
 'r4': (0.9600772353397823, 0.03992276466021776),
 'r5': (0.0011309404790580402, 0.998869059520942),
 'r6': (0.900148428675396, 0.099851571324604),
 'r7': (0.13037362396293917, 0.8696263760370608),
 'r8': (0.49406553049687485, 0.5059344695031252),
 'r9': (0.8464211309285306, 0.1535788690714695),
 'r10': (0.5337915699721013, 0.4662084300278987),
 'r11': (0.957012071468477, 0.04298792853152301),
 'r12': (0.9017510875213329, 0.0982489124786671),
 'r13': (0.9724594389292762, 0.027540561070723776),
 'r14': (0.9708343227627803, 0.0291656772372198),
 'r15': (0.011552728050316152, 0.9884472719496838),
 'r16': (0.9981927826621926, 0.001807217337807407),
 'r17': (0.9936994581104033, 0.006300541889596699),
 'r18': (0.003973816797989138, 0.9960261832020109),
 'r19': (0.016500236150219116, 0.9834997638497809),
 'r20': (0.9928896191707224, 0.00711038082

Now we do all the timestep recursively 


In [151]:
first_state = data.head(1)
first_room_dict = first_state.iloc[:,17:].to_dict()
first_room_state = dict_to_state(first_room_dict, "room")

first_censor_dict =  data_sample.iloc[:,1:9].to_dict()
first_censor_state = dict_to_state(first_censor_dict, "censor")

In [152]:
all_room_predict(first_room_state,first_censor_state, rooms_df,censor_df, [1,9]  )

{'r1': (0.9986244715036079, 0.001375528496392076),
 'r2': (0.6922010401567693, 0.30779895984323075),
 'r3': (0.8732633933311361, 0.1267366066688639),
 'r4': (0.9600772353397823, 0.03992276466021776),
 'r5': (0.9886982664271079, 0.011301733572892068),
 'r6': (0.9771555875354426, 0.022844412464557422),
 'r7': (0.9954139822229566, 0.004586017777043351),
 'r8': (0.6320732949188523, 0.3679267050811477),
 'r9': (0.9153211316891529, 0.08467886831084712),
 'r10': (0.9999992618258357, 7.381741643118151e-07),
 'r11': (0.9988210935150521, 0.0011789064849479496),
 'r12': (0.0030587850915568815, 0.9969412149084431),
 'r13': (0.9678923909972318, 0.03210760900276821),
 'r14': (0.9708343227627803, 0.0291656772372198),
 'r15': (0.9996939216388571, 0.0003060783611429342),
 'r16': (0.9998517166554253, 0.00014828334457477882),
 'r17': (0.9832359406400615, 0.016764059359938556),
 'r18': (0.9721484193778249, 0.027851580622175045),
 'r19': (0.9984502374200425, 0.0015497625799575092),
 'r20': (0.9457462152783

In [164]:
list_prob =[]
for idx, value in data.time.items():
    curr_censor = data.iloc[idx,1:9].to_dict()
    curr_censor_state = dict_to_state(curr_censor, "censor")
    
    if idx == 0:
        current_room_state = first_room_state
    else: 
        current_room_state = list_prob[idx-1]
    
    room_predict = all_room_predict(current_room_state,curr_censor_state, rooms_df,censor_df, [1,9]  )
    list_prob.append(room_predict)

In [165]:
pro_df= pd.DataFrame(list_prob)
#pro_df.to_csv('predict2.csv')

In [166]:
def turn_on(prob, theta = 0.7):
    turn_on = 1 
    if prob[0] > theta:
        turn_on = 0
    return(turn_on)

def have_people(number):
    turn_on = 1 
    if number < 0.1:
        turn_on = 0
    return (turn_on)

In [167]:
decision_df = pro_df.applymap(turn_on)


In [160]:
true_data = data.iloc[:,17:].applymap(have_people)
def calculate_cost (light,people):
    if light == 1:
        cost = 1 
    if light == 0:
        cost = people*4
    return(cost)

In [168]:
total_error = []
for idx, value in data.time.items():
    error_list = true_data.iloc[idx,:] - decision_df.iloc[idx,:]
    errors = sum(x != 0 for x in error_list[:35])
    total_error.append(errors)

total_error
error_rate = [i/35 for i in total_error]
error_rate

[0.05714285714285714,
 0.11428571428571428,
 0.2,
 0.2571428571428571,
 0.2571428571428571,
 0.3142857142857143,
 0.3142857142857143,
 0.2857142857142857,
 0.34285714285714286,
 0.3142857142857143,
 0.45714285714285713,
 0.45714285714285713,
 0.45714285714285713,
 0.4,
 0.42857142857142855,
 0.5142857142857142,
 0.4857142857142857,
 0.45714285714285713,
 0.42857142857142855,
 0.42857142857142855,
 0.45714285714285713,
 0.45714285714285713,
 0.45714285714285713,
 0.42857142857142855,
 0.4857142857142857,
 0.42857142857142855,
 0.4,
 0.37142857142857144,
 0.37142857142857144,
 0.37142857142857144,
 0.4,
 0.4,
 0.37142857142857144,
 0.34285714285714286,
 0.34285714285714286,
 0.34285714285714286,
 0.37142857142857144,
 0.37142857142857144,
 0.37142857142857144,
 0.3142857142857143,
 0.3142857142857143,
 0.34285714285714286,
 0.2857142857142857,
 0.2857142857142857,
 0.2857142857142857,
 0.3142857142857143,
 0.4,
 0.3142857142857143,
 0.22857142857142856,
 0.3142857142857143,
 0.2571428571

In [176]:
total_cost = []
for idx, value in data.time.items():
    cost_list = [calculate_cost(i,j) for i,j in zip(decision_df.iloc[idx,:35],data.iloc[idx,17:])]
    cost = sum(cost_list)
    total_cost.append(cost)

In [177]:
sum(total_cost)

114531

In [178]:
total_cost

[4,
 6,
 10,
 11,
 11,
 19,
 12,
 11,
 17,
 13,
 33,
 46,
 46,
 35,
 45,
 55,
 62,
 71,
 70,
 70,
 70,
 70,
 70,
 63,
 65,
 63,
 62,
 61,
 61,
 61,
 65,
 61,
 65,
 57,
 57,
 61,
 65,
 65,
 65,
 56,
 56,
 51,
 49,
 50,
 50,
 48,
 51,
 50,
 45,
 50,
 46,
 48,
 53,
 44,
 47,
 46,
 51,
 46,
 46,
 47,
 50,
 51,
 50,
 51,
 47,
 51,
 48,
 51,
 62,
 60,
 60,
 59,
 56,
 56,
 55,
 51,
 52,
 53,
 53,
 51,
 52,
 55,
 48,
 47,
 47,
 49,
 47,
 48,
 47,
 46,
 47,
 47,
 42,
 43,
 43,
 41,
 41,
 46,
 48,
 44,
 42,
 38,
 46,
 43,
 41,
 40,
 29,
 30,
 30,
 30,
 31,
 37,
 38,
 38,
 35,
 36,
 35,
 34,
 34,
 34,
 30,
 33,
 30,
 29,
 34,
 35,
 30,
 31,
 31,
 31,
 33,
 34,
 33,
 34,
 33,
 33,
 33,
 33,
 33,
 33,
 35,
 35,
 40,
 36,
 43,
 40,
 34,
 38,
 34,
 35,
 38,
 35,
 35,
 40,
 40,
 40,
 41,
 40,
 40,
 36,
 36,
 32,
 36,
 36,
 41,
 40,
 40,
 41,
 36,
 37,
 37,
 36,
 40,
 39,
 40,
 36,
 35,
 31,
 27,
 27,
 27,
 29,
 29,
 23,
 30,
 35,
 32,
 32,
 29,
 31,
 32,
 31,
 33,
 34,
 26,
 31,
 35,
 34,
 42,
 42,
 4

In [79]:
def prob_have_people(state,trans):
    "trans is the dictionary from rooms_tran; state is a dictionary with room as key and (p(0), p(1)) as value "
    prob_have = trans['(1,1)']*state[1] +  trans["(1,0)"]*state[0] 
    return (prob_have)
    
def prob_no_people(state,trans):    
    prob_no = trans["(0,1)"]*state[1] +  trans["(0,0)"]*state[0] 
    return (prob_no)


state_tem = {'c1': (1.0,0)}
trans_tem = room_prob.loc[ room_prob['previous_room'] == 'c1',:].to_dict()
print(trans_tem)

print(prob_have_people(state_tem, trans_tem))

{'Unnamed: 0': {193: 193}, 'previous_room': {193: 'c1'}, 'current_room': {193: 'r25'}, '(1, 1)': {193: 0.365505425}, '(1, 0)': {193: 0.634494575}, '(0, 1)': {193: 0.334360555}, '(0, 0)': {193: 0.665639445}}


KeyError: '(1,1)'

In [57]:
for rooms in room_prob['previous_room']:
    room_tran = room_prob.loc[ room_prob['previous_room'] == rooms,:].to_dict
    room_state = current_state['rooms']

0.3655

In [ ]:
def state_generator(room):
    state_dict = {'dom': (room,),    
                  'table': odict([
        ((1,), 0.0),
        ((0,), 1.0),
    ])}
    return (state_dict)
state_info = []
for rooms in room_prob['previous_room']:
    state_info.append(state_generator(rooms))
    
state_info

In [ ]:
def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))

def prob(factor, *entry):
    """
    argument 
    `factor`, a dictionary of domain and probability values,
    `entry`, a list of values, one for each variable in the same order as specified in the factor domain.
    
    Returns p(entry)
    """

    return factor['table'][entry]     # insert your code here, 1 line   

def join(f1, f2, outcomeSpace):
    """
    argument 
    `f1`, first factor to be joined.
    `f2`, second factor to be joined.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor with a join of f1 and f2
    """
    
    # First, we need to determine the domain of the new factor. It will be union of the domain in f1 and f2
    # But it is important to eliminate the repetitions
    common_vars = list(f1['dom']) + list(set(f2['dom']) - set(f1['dom']))
    
    # We will build a table from scratch, starting with an empty list. Later on, we will transform the list into a odict
    table = list()
    
    # Here is where the magic happens. The product iterator will generate all combinations of varible values 
    # as specified in outcomeSpace. Therefore, it will naturally respect observed values
    for entries in product(*[outcomeSpace[node] for node in common_vars]):
        
        # We need to map the entries to the domain of the factors f1 and f2
        entryDict = dict(zip(common_vars, entries))
        f1_entry = (entryDict[var] for var in f1['dom'])
        f2_entry = (entryDict[var] for var in f2['dom'])
        
        # Insert your code here
        p1 = prob(f1, *f1_entry)           # Use the fuction prob to calculate the probability in factor f1 for entry f1_entry 
        p2 = prob(f2, *f2_entry)           # Use the fuction prob to calculate the probability in factor f2 for entry f2_entry 
        
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, p1 * p2))
    return {'dom': tuple(common_vars), 'table': odict(table)}


def marginalize(f, var, outcomeSpace):
    """
    argument 
    `f`, factor to be marginalized.
    `var`, variable to be summed out.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor f' with dom(f') = dom(f) - {var}
    """    
    
    # Let's make a copy of f domain and convert it to a list. We need a list to be able to modify its elements
    new_dom = list(f['dom'])
    
    new_dom.remove(var)            # Remove var from the list new_dom by calling the method remove(). 1 line
    table = list()                 # Create an empty list for table. We will fill in table from scratch. 1 line
    for entries in product(*[outcomeSpace[node] for node in new_dom]):
        s = 0;                     # Initialize the summation variable s. 1 line

        # We need to iterate over all possible outcomes of the variable var
        for val in outcomeSpace[var]:
            # To modify the tuple entries, we will need to convert it to a list
            entriesList = list(entries)
            # We need to insert the value of var in the right position in entriesList
            entriesList.insert(f['dom'].index(var), val)
            
            p = prob(f, *tuple(entriesList))     # Calculate the probability of factor f for entriesList. 1 line
            s = s + p                            # Sum over all values of var by accumulating the sum in s. 1 line
            
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, s))
    return {'dom': tuple(new_dom), 'table': odict(table)}

In [ ]:
c1_trans = {
    'dom': ('previous', 'current'), 
    'table': odict([
        (((1,1),), 0.3655),
        (((1,0),), 0.6345),
        (((0,1),), 0.3344),
        (((0,0),), 0.6656),
    ])
}
state_current = {'dom': ('c1',), 'table': odict([((1,), 0.0), ((0,), 1.0)])}

prob(c1_trans , (1,1))
current_state = {'c1': (0,1)}


In [ ]:
def prob_have_people(state,trans):
    prob_have = prob(trans, (1,1))*state[1] +  prob(trans, (1,0))*state[0] 
    return (prob_have)

def prob_no_people(state,trans):    
    prob_no = prob(trans, (0,1))*prob(state,(1)) +  prob(trans, (0,0))*prob(state,(0))
    return (prob_no)

print(prob_have_people(state_current,c1_trans))
print(prob_no_people(state_current,c1_trans))